In [ ]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
import gdown
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import tensorflow.keras as keras
import tensorflow as tf
import gc
from google.colab import drive
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import shutil
drive.mount('/content/drive')

gc.enable()

Mounted at /content/drive


In [ ]:
# Download all the necessary files for the BirdCLEF2023 competition


! gdown  1csRtTkSjkr4zi02A7hh92Vd3-q8bRu9U # https://drive.google.com/file/d/1csRtTkSjkr4zi02A7hh92Vd3-q8bRu9U/view?usp=share_link

! gdown  171Dt710QatK8_fpGOowyOCq0--8iKZNi # https://drive.google.com/file/d/171Dt710QatK8_fpGOowyOCq0--8iKZNi/view?usp=share_link

! gdown  1QZtXYvPpZ9PgvWhS7AjPiKCWmSSx2N3_ # https://drive.google.com/file/d/1QZtXYvPpZ9PgvWhS7AjPiKCWmSSx2N3_/view?usp=share_link

! gdown  1wJOwvbRRhV7-jbCunlT6YI_yuPpRu8mn # https://drive.google.com/file/d/1wJOwvbRRhV7-jbCunlT6YI_yuPpRu8mn/view?usp=share_link

! gdown  1cbB3f6kFGUshonoGIa9C4tUQdZm1UYsY # https://drive.google.com/file/d/1cbB3f6kFGUshonoGIa9C4tUQdZm1UYsY/view?usp=share_link

! gdown  1fhEzrtHzjT2J_LZM-vIOemXjzmKtnxXc # https://drive.google.com/file/d/1fhEzrtHzjT2J_LZM-vIOemXjzmKtnxXc/view?usp=drive_link

Downloading...
From: https://drive.google.com/uc?id=1csRtTkSjkr4zi02A7hh92Vd3-q8bRu9U
To: /content/audio_bird_name.csv
100% 148k/148k [00:00<00:00, 123MB/s]
Downloading...
From: https://drive.google.com/uc?id=171Dt710QatK8_fpGOowyOCq0--8iKZNi
To: /content/audio_data_Y.csv
100% 76.7k/76.7k [00:00<00:00, 46.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QZtXYvPpZ9PgvWhS7AjPiKCWmSSx2N3_
To: /content/eBird_Taxonomy_v2021.csv
100% 2.01M/2.01M [00:00<00:00, 18.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1wJOwvbRRhV7-jbCunlT6YI_yuPpRu8mn
To: /content/sample_submission.csv
100% 3.74k/3.74k [00:00<00:00, 17.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1cbB3f6kFGUshonoGIa9C4tUQdZm1UYsY
To: /content/train_metadata.csv
100% 3.60M/3.60M [00:00<00:00, 236MB/s]
Downloading...
From: https://drive.google.com/uc?id=1fhEzrtHzjT2J_LZM-vIOemXjzmKtnxXc
To: /content/augmented_data_Y_50.csv
100% 24.4k/24.4k [00:00<00:00, 54.7MB/s]


In [ ]:
# Download the splitted X audio data from the Google Drive which has been saved.
# This jupyter code generated those files: https://colab.research.google.com/drive/14WKdPt7H-psYb-hjG1ftCPeH9wevuJ1A

def download_files(file_ids, prefix, file_type="npy"):
    for i, file_id in enumerate(file_ids):
        destination = f'{prefix}_X{i + 1}.{file_type}'
        url = f'https://drive.google.com/uc?id={file_id}'
        gdown.download(url, destination, quiet=False)


# Define the Google Drive file ID
file_id_audio = [
            '1gSu-EkrSYmjxYg18DLKP9VcQs--QqQ0l',
           '1ZLSaUMdrTLjZYk-VUKHALkJNqEHOMUQb',
           '1tW10PTV8wWG0LNjI7tQCxodwDJBofWxS',
           '1s32-Ou0uexEoej3LWdvV3c4sExb7bdM7',
           '1cyExp5s0eNlw3o23vXi50C6eLCFzp-Md',
           '18fLcutxetAPlXIsjO5E8vt5Z96p91uVo',
           '1hUw5dfMJf_xp0V8YK8iUedMLETYi0Box',
           '15t09xUyAGCVfTPJIEFTw2TWlxRk55TJB',
           '1QtPz9aML3FkflFE9kf4fggQaYi2P_vV7',
           '1x_H-RrVesqHbx0CnH0ee2T-W2NnHr-so',
           ]

# Define the Google Drive file ID
file_id_augmented = [
            '1wBIZLPuCIbEGpVBntP3wEVe_qx1TXN9f', # https://drive.google.com/file/d/1wBIZLPuCIbEGpVBntP3wEVe_qx1TXN9f/view?usp=drive_link
            '1DT8bThsLgNRy3S0fEiKeJciG_PWYeVXS', # https://drive.google.com/file/d/1DT8bThsLgNRy3S0fEiKeJciG_PWYeVXS/view?usp=drive_link
            '15VMweJEogukNmmddHWWDP7Xh24kuirKf', # https://drive.google.com/file/d/15VMweJEogukNmmddHWWDP7Xh24kuirKf/view?usp=drive_link
            '1biiGgxvu_nOK1Y92DhjqaHsK9wiyAJkX', # https://drive.google.com/file/d/1biiGgxvu_nOK1Y92DhjqaHsK9wiyAJkX/view?usp=drive_link

            ]


# Download audio data
download_files(file_id_audio, "audio_data")

# Download augmented data
download_files(file_id_augmented, "augmented_data")

Downloading...
From: https://drive.google.com/uc?id=1gSu-EkrSYmjxYg18DLKP9VcQs--QqQ0l
To: /content/audio_data_X1.npy
100%|██████████| 272M/272M [00:02<00:00, 104MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZLSaUMdrTLjZYk-VUKHALkJNqEHOMUQb
To: /content/audio_data_X2.npy
100%|██████████| 272M/272M [00:05<00:00, 51.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1tW10PTV8wWG0LNjI7tQCxodwDJBofWxS
To: /content/audio_data_X3.npy
100%|██████████| 272M/272M [00:04<00:00, 61.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1s32-Ou0uexEoej3LWdvV3c4sExb7bdM7
To: /content/audio_data_X4.npy
100%|██████████| 272M/272M [00:02<00:00, 104MB/s]
Downloading...
From: https://drive.google.com/uc?id=1cyExp5s0eNlw3o23vXi50C6eLCFzp-Md
To: /content/audio_data_X5.npy
100%|██████████| 272M/272M [00:02<00:00, 97.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=18fLcutxetAPlXIsjO5E8vt5Z96p91uVo
To: /content/audio_data_X6.npy
100%|██████████| 272M/272M [00:02<00:00, 106M

In [ ]:
# Function to load and concatenate data from files
def load_and_concatenate_data(base_name, num_files):
    concatenated_data = None
    for i in range(1, num_files + 1):
        file_name = f'{base_name}{i}.npy'
        loaded_array = np.load(file_name)
        if concatenated_data is None:
            concatenated_data = loaded_array
        else:
            concatenated_data = np.concatenate((concatenated_data, loaded_array), axis=0)
    return concatenated_data

# Load and concatenate audio data
audio_data_X = load_and_concatenate_data('audio_data_X', 10) # len(file_id_audio)
augmented_data_X = load_and_concatenate_data('augmented_data_X', 4) # len(file_id_augmented)



In [ ]:
# Load Y (the id column, int data type) and the Y_bird_name (the name column)
audio_data_Y = np.loadtxt("/content/audio_data_Y.csv", delimiter=',',dtype=int)
augmented_data_Y = np.loadtxt("/content/augmented_data_Y_50.csv", delimiter=',',dtype=int)
audio_data_Y -= 1
augmented_data_Y -= 1

# audio_data_Y = np.concatenate((audio_data_Y, augmented_data_Y), axis=0)
# del augmented_data_Y

audio_data_Y_bird_name = np.genfromtxt("/content/audio_bird_name.csv", delimiter=',', dtype='str').astype('object')

# Generate an array of IDs for each bird from 1 to number of the unique birds
ids = np.arange(0, len(np.unique(audio_data_Y_bird_name)))
# Add the ID column as a new dimension (column)
audio_data_Y_unique_bird_name_with_id = np.column_stack((ids, np.unique(audio_data_Y_bird_name)))

# The unique bird that contains each bird with ID
print(audio_data_Y_unique_bird_name_with_id)

[[0 'abethr1']
 [1 'abhori1']
 [2 'abythr1']
 [3 'afbfly1']
 [4 'afdfly1']
 [5 'afecuc1']
 [6 'affeag1']
 [7 'afgfly1']
 [8 'afghor1']
 [9 'afmdov1']
 [10 'afpfly1']
 [11 'afpkin1']
 [12 'afpwag1']
 [13 'afrgos1']
 [14 'afrgrp1']
 [15 'afrjac1']
 [16 'afrthr1']
 [17 'amesun2']
 [18 'augbuz1']
 [19 'bagwea1']
 [20 'barswa']
 [21 'bawhor2']
 [22 'bawman1']
 [23 'bcbeat1']
 [24 'beasun2']
 [25 'bkctch1']
 [26 'bkfruw1']
 [27 'blacra1']
 [28 'blacuc1']
 [29 'blakit1']
 [30 'blaplo1']
 [31 'blbpuf2']
 [32 'blcapa2']
 [33 'blfbus1']
 [34 'blhgon1']
 [35 'blhher1']
 [36 'blksaw1']
 [37 'blnmou1']
 [38 'blnwea1']
 [39 'bltapa1']
 [40 'bltbar1']
 [41 'bltori1']
 [42 'blwlap1']
 [43 'brcale1']
 [44 'brcsta1']
 [45 'brctch1']
 [46 'brcwea1']
 [47 'brican1']
 [48 'brobab1']
 [49 'broman1']
 [50 'brosun1']
 [51 'brrwhe3']
 [52 'brtcha1']
 [53 'brubru1']
 [54 'brwwar1']
 [55 'bswdov1']
 [56 'btweye2']
 [57 'bubwar2']
 [58 'butapa1']
 [59 'cabgre1']
 [60 'carcha1']
 [61 'carwoo1']
 [62 'categr']
 [63

In [ ]:
# Calculate the unique values and their frequencies
unique_bird, counts = np.unique(np.concatenate((audio_data_Y, augmented_data_Y), axis=0), return_counts=True)
plt.figure(figsize=(50, 30))

# Create a bar plot (histogram)
plt.bar(audio_data_Y_unique_bird_name_with_id[:,1], counts, width=0.3)

# Set labels for the axes
plt.xlabel('Unique Birds', fontsize=30)
plt.ylabel('The number of audio for the birds', fontsize=30)
plt.xticks(rotation=90)
yticks_positions = np.append(np.arange(0, 100, 10),np.arange(100, max(counts) + 1, 50))  # Adjust the step (5) as needed
plt.yticks(yticks_positions, fontsize=20)
# Show the plot
plt.show()

In [ ]:
validation_percent = 0.4
validation_index = len(audio_data_Y) - int(validation_percent * (len(audio_data_Y) + len(augmented_data_Y)))
# validation_index = (len(audio_data_Y) + len(augmented_data_Y)) - validation_size

permutation = np.random.permutation(len(audio_data_Y))

audio_data_X = audio_data_X[permutation]
audio_data_Y = audio_data_Y[permutation]


permutation = np.random.permutation(len(augmented_data_Y))

augmented_data_X = augmented_data_X[permutation]
augmented_data_Y = augmented_data_Y[permutation]


In [ ]:
X_train = np.concatenate((augmented_data_X, audio_data_X[:validation_index]), axis=0)
y_train = np.concatenate((augmented_data_Y, audio_data_Y[:validation_index]), axis=0)

X_validation =  audio_data_X[validation_index:]
y_validation =  audio_data_Y[validation_index:]

X_test = X_validation[(len(X_validation)//2):]
y_test = y_validation[(len(y_validation)//2):]

X_validation = X_validation[:(len(X_validation)//2)]
y_validation = y_validation[:(len(y_validation)//2)]


permutation = np.random.permutation(len(y_train))

X_train = X_train[permutation]
y_train = y_train[permutation]

augmented_data_Y.shape

del augmented_data_X
del augmented_data_Y
del audio_data_X
del audio_data_Y

In [ ]:
min_value_train, max_value_train = np.min(X_train), np.max(X_train)
min_value_test, max_value_test = np.min(X_test), np.max(X_test)
min_value_validation, max_value_validation = np.min(X_validation), np.max(X_validation)


X_train = (X_train - min_value_train) / (max_value_train - min_value_train)
X_test = (X_test - min_value_test) / (max_value_test - min_value_test)
X_validation = (X_validation - min_value_validation) / (max_value_validation - min_value_validation)

X_train= X_train.reshape(-1,128,313,1)
X_test= X_test.reshape(-1,128,313,1)
X_validation= X_validation.reshape(-1,128,313,1)


In [ ]:
def build_model(input_shape):
  """Generates CNN model

  :param input_shape (tuple): Shape of input set
  :return model: CNN model
  """

  # build network topology
  model = keras.Sequential()

  # 1st conv layer
  model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
  model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
  model.add(keras.layers.BatchNormalization())

  # 2nd conv layer
  model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
  model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
  model.add(keras.layers.BatchNormalization())


  # flatten output and feed it into dense layer
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(64, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
  model.add(keras.layers.Dropout(0.5))

  # output layer
  model.add(keras.layers.Dense(264, activation='softmax'))

  return model

In [ ]:
# create network
input_shape = (X_train.shape[1], X_train.shape[2], 1)
model = build_model(input_shape)

# compile model
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 311, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 156, 32)       0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 63, 156, 32)       128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 154, 32)       9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 31, 77, 32)        0         
 g2D)                                                            
                                                        

In [ ]:
patience = 20
early_stopping=EarlyStopping(patience=patience, verbose=2)
checkpointer=ModelCheckpoint(filepath='weights.hdf5', save_best_only=True, verbose=2)

In [ ]:

history = model.fit(X_train[:], y_train[:],
                    validation_data=(X_validation[:], y_validation[:]),
                    batch_size=128,
                    epochs=1000,
                    callbacks=[checkpointer, early_stopping])

Epoch 1/1000
105/105 [==============================] - ETA: 0s - loss: 5.6798 - accuracy: 0.0200
Epoch 1: val_loss improved from inf to 5.79405, saving model to weights.hdf5
105/105 [==============================] - 30s 153ms/step - loss: 5.6798 - accuracy: 0.0200 - val_loss: 5.7941 - val_accuracy: 0.0027
Epoch 2/1000


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


104/105 [============================>.] - ETA: 0s - loss: 5.3736 - accuracy: 0.0493
Epoch 2: val_loss did not improve from 5.79405
105/105 [==============================] - 11s 109ms/step - loss: 5.3737 - accuracy: 0.0492 - val_loss: 6.1281 - val_accuracy: 0.0027
Epoch 3/1000
104/105 [============================>.] - ETA: 0s - loss: 5.0190 - accuracy: 0.0979
Epoch 3: val_loss did not improve from 5.79405
105/105 [==============================] - 11s 105ms/step - loss: 5.0185 - accuracy: 0.0980 - val_loss: 6.2072 - val_accuracy: 0.0029
Epoch 4/1000
104/105 [============================>.] - ETA: 0s - loss: 4.5652 - accuracy: 0.1552
Epoch 4: val_loss did not improve from 5.79405
105/105 [==============================] - 12s 111ms/step - loss: 4.5650 - accuracy: 0.1552 - val_loss: 5.8420 - val_accuracy: 0.0491
Epoch 5/1000
104/105 [============================>.] - ETA: 0s - loss: 4.1280 - accuracy: 0.2183
Epoch 5: val_loss improved from 5.79405 to 5.42445, saving model to weights.hd

KeyboardInterrupt: ignored